In [12]:
import mysql.connector
import pandas as pd
from IPython.display import Markdown, display

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="secret",
  port="3306",
  database="serlo"
)

def printmd(string):
    display(Markdown(string))

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def query_number(sql):
    return query(sql)[0][0]

In [13]:
year = 2024

printmd(f"# Auswertung für das Jahr {year}")

# Auswertung für das Jahr 2024

In [14]:
count = query_number("select count(*) from event where YEAR(event.date) = %s and event.event_type_id IN (5);" % year)

printmd(f"#### Anzahl Bearbeitungen: {count}")

#### Anzahl Bearbeitungen: 7031

In [15]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "article" and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "article" and year(entity.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Artikel in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Artikel: {count}")

#### Anzahl neuer Artikel in diesem Jahr: 112

#### Aktueller Bestand von Artikel: 3927

In [16]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "course-page" and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "course-page"  and year(entity.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Kursseiten in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Kursen: {count}")

#### Anzahl neuer Kursseiten in diesem Jahr: 51

#### Aktueller Bestand von Kursen: 4274

In [17]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name in ("text-exercise", "text-exercise-group") and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name in ("text-exercise", "text-exercise-group") and year(entity.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Aufgaben in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Aufgaben: {count}")

#### Anzahl neuer Aufgaben in diesem Jahr: 1564

#### Aktueller Bestand von Aufgaben: 9778

In [18]:
new_count = query_number("""
    select count(*) from comment
    join uuid on comment.id = uuid.id
    where uuid.trashed = 0 and year(comment.date) = %s
""" % year)
count = query_number("""
    select count(*) from comment
    join uuid on comment.id = uuid.id
    where uuid.trashed = 0 and year(comment.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Kommentare in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Kommentaren: {count}")

#### Anzahl neuer Kommentare in diesem Jahr: 456

#### Aktueller Bestand von Kommentaren: 9703

In [19]:
printmd("### Liste von Autor:innen der Anzahl ihrer Bearbeitung Bearbeitungen")
pd.set_option('display.max_rows', None)
df = pd.read_sql("""
    select user.username, count(event.actor_id) as edits from event
    join user on user.id = event.actor_id
    where year(event.date) = %s
    group by event.actor_id
    order by edits desc
""" % year, db)
df.to_csv("/tmp/authors.csv")
df

### Liste von Autor:innen der Anzahl ihrer Bearbeitung Bearbeitungen

<ipython-input-19-6db7bfbe936b>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


,username,edits
0,Felix_Eccardt,4347
1,Karin,2191
2,botho,1900
3,Kowalsky,1383
4,Peter,1313
5,anne22,1121
6,KlausB,771
7,Alexandra-Wolfram,758
8,dal,583
9,Sascha_Lill_95,486
